# Static Data Acquisition for Wflow Niger Basin Model

**Objective:** Download all required static datasets to build a Wflow SBM model for the Upper Niger Basin, independent of any external catalog.

## Required Datasets
| Dataset | Source | Variables |
|---------|--------|----------|
| **DEM** | NASA SRTM 90m | Elevation |
| **Flow Direction** | Derived from DEM | LDD |
| **Soil** | SoilGrids 250m | Sand/Clay fractions |
| **Land Cover** | Placeholder | LULC classes |
| **LAI** | Synthetic climatology | Leaf Area Index |
| **Forcing** | ERA5-Land | Precip, Temp, PET |

---


In [ ]:
# 1. Setup
import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import xarray as xr
import pandas as pd

# Output directory
DATA_DIR = Path("../data/static")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Study Area: Upper Niger Basin
BBOX = [-10.5, 9.5, -6.5, 13.0]  # [West, South, East, North]
RES = 0.00833  # ~1km resolution

print(f"Study area: {BBOX}")
print(f"Resolution: {RES} degrees")
print(f"Output: {DATA_DIR.resolve()}")


## 2. DEM Acquisition (SRTM via elevation package)


In [ ]:
# Install elevation package if needed
%pip install elevation rasterio pyflwdir -q

import elevation
import rasterio

dem_path = DATA_DIR / "dem_niger.tif"

if not dem_path.exists():
    print("Downloading SRTM DEM...")
    elevation.clip(bounds=BBOX, output=str(dem_path), product='SRTM3')
    elevation.clean()
    print(f"DEM saved: {dem_path}")
else:
    print(f"DEM exists: {dem_path}")

# Verify
with rasterio.open(dem_path) as src:
    print(f"Shape: {src.shape}")
    print(f"CRS: {src.crs}")
    print(f"Resolution: {src.res}")


## 3. Flow Direction (LDD) from DEM

Using `pyflwdir` to derive D8 flow direction.


In [ ]:
import pyflwdir

# Load DEM
with rasterio.open(dem_path) as src:
    dem = src.read(1)
    transform = src.transform
    crs = src.crs
    nodata = src.nodata if src.nodata else -9999

# Fill depressions and derive flow direction
print("Deriving flow direction...")
dem_filled = pyflwdir.dem.fill_depressions(dem, nodata=nodata)
flw = pyflwdir.from_dem(dem_filled, nodata=nodata, transform=transform, latlon=True)

# Get D8 flow direction
flwdir_d8 = flw.to_array('d8')

# Save as GeoTIFF
ldd_path = DATA_DIR / "flwdir_niger.tif"
with rasterio.open(ldd_path, 'w', driver='GTiff', height=flwdir_d8.shape[0],
                   width=flwdir_d8.shape[1], count=1, dtype=flwdir_d8.dtype,
                   crs=crs, transform=transform, nodata=0) as dst:
    dst.write(flwdir_d8, 1)
print(f"Flow direction saved: {ldd_path}")

# Derive upstream area
uparea = flw.upstream_area('km2')
uparea_path = DATA_DIR / "uparea_niger.tif"
with rasterio.open(uparea_path, 'w', driver='GTiff', height=uparea.shape[0],
                   width=uparea.shape[1], count=1, dtype='float32',
                   crs=crs, transform=transform, nodata=-9999) as dst:
    dst.write(uparea.astype('float32'), 1)
print(f"Upstream area saved: {uparea_path}")


## 4. Land Slope from DEM


In [ ]:
# Calculate slope (m/m)
dy, dx = np.gradient(dem_filled)
cell_size = abs(transform[0]) * 111000  # degrees to meters
slope = np.sqrt((dx/cell_size)**2 + (dy/cell_size)**2)
slope = np.clip(slope, 0.0001, 1.0)  # Avoid zero slopes

slope_path = DATA_DIR / "slope_niger.tif"
with rasterio.open(slope_path, 'w', driver='GTiff', height=slope.shape[0],
                   width=slope.shape[1], count=1, dtype='float32',
                   crs=crs, transform=transform, nodata=-9999) as dst:
    dst.write(slope.astype('float32'), 1)
print(f"Slope saved: {slope_path}")


## 5. LAI Monthly Climatology (West African Savanna)


In [ ]:
# Create monthly LAI climatology for savanna (West African monsoon pattern)
# Peak LAI in Aug-Sep (wet season), minimum in Jan-Mar (dry season)
lai_monthly = {1: 0.5, 2: 0.4, 3: 0.5, 4: 0.8, 5: 1.2, 6: 1.8,
               7: 2.5, 8: 3.0, 9: 2.8, 10: 2.0, 11: 1.2, 12: 0.7}

lai_stack = [np.full(dem.shape, lai_monthly[m], dtype='float32') for m in range(1, 13)]

# Get coordinates
with rasterio.open(dem_path) as src:
    left, bottom, right, top = src.bounds
    lons = np.linspace(left + abs(transform[0])/2, right - abs(transform[0])/2, src.width)
    lats = np.linspace(top - abs(transform[4])/2, bottom + abs(transform[4])/2, src.height)

times = pd.date_range('2019-01-15', periods=12, freq='MS') + pd.Timedelta(days=14)

ds_lai = xr.Dataset(
    {"LAI": (["time", "y", "x"], np.stack(lai_stack))},
    coords={"time": times, "y": lats, "x": lons}
)
ds_lai["LAI"].attrs = {"units": "m2/m2", "long_name": "Leaf Area Index"}

lai_path = DATA_DIR / "lai_niger.nc"
ds_lai.to_netcdf(lai_path)
print(f"LAI saved: {lai_path}")


## 6. Create Combined Static Maps NetCDF


In [ ]:
# Load all rasters
def load_raster(path):
    with rasterio.open(path) as src:
        return src.read(1)

flwdir_data = load_raster(ldd_path)
slope_data = load_raster(slope_path)
uparea_data = load_raster(uparea_path)

# Create river mask (upstream area > 10 km2)
river_mask = (uparea > 10).astype('uint8')

# Create subcatchment IDs (simplified)
subcatch = np.zeros_like(dem, dtype='int32')
subcatch[river_mask == 1] = 1

# Create static dataset
ds_static = xr.Dataset({
    "wflow_dem": (["y", "x"], dem_filled.astype('float32')),
    "wflow_ldd": (["y", "x"], flwdir_data.astype('uint8')),
    "wflow_river": (["y", "x"], river_mask),
    "wflow_subcatch": (["y", "x"], subcatch),
    "wflow_uparea": (["y", "x"], uparea.astype('float32')),
    "Slope": (["y", "x"], slope_data.astype('float32')),
}, coords={"y": lats, "x": lons})

ds_static.attrs["crs"] = "EPSG:4326"
ds_static["wflow_dem"].attrs = {"units": "m", "long_name": "Elevation"}
ds_static["Slope"].attrs = {"units": "m/m", "long_name": "Land slope"}

static_path = DATA_DIR / "staticmaps_niger.nc"
ds_static.to_netcdf(static_path)
print(f"Static maps saved: {static_path}")
print(ds_static)


## 7. Visualize Static Data


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
plt.suptitle("Niger Basin Static Data", fontsize=14)

ds_static["wflow_dem"].plot(ax=axes[0,0], cmap="terrain")
axes[0,0].set_title("DEM (m)")

ds_static["wflow_ldd"].plot(ax=axes[0,1], cmap="viridis")
axes[0,1].set_title("Flow Direction (D8)")

ds_static["Slope"].plot(ax=axes[0,2], cmap="YlOrRd")
axes[0,2].set_title("Slope (m/m)")

np.log10(ds_static["wflow_uparea"] + 1).plot(ax=axes[1,0], cmap="Blues")
axes[1,0].set_title("log10(Upstream Area km²)")

ds_static["wflow_river"].plot(ax=axes[1,1], cmap="Blues")
axes[1,1].set_title("River Network")

ds_lai["LAI"].mean(dim="time").plot(ax=axes[1,2], cmap="Greens")
axes[1,2].set_title("Mean LAI")

plt.tight_layout()
plt.show()

print("\nData files created:")
for f in DATA_DIR.glob("*"):
    size_mb = f.stat().st_size / 1024 / 1024
    print(f"  {f.name}: {size_mb:.2f} MB")
